In [2]:
import pandas as pd
import re

# Nom du fichier à lire
file_name = "Discussion WhatsApp avec Ruben.txt"

# Préparer les listes pour stocker les données extraites
dates = []
times = []
names = []
messages = []

# Regex pour extraire les données
pattern = re.compile(r"(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}) - (.*?): (.*)")

# Lire le fichier ligne par ligne et extraire les données
with open(file_name, "r", encoding="utf-8") as file:
    for line in file:
        match = pattern.search(line)
        if match:
            date, time, name, message = match.groups()
            dates.append(date)
            times.append(time)
            names.append(name)
            messages.append(message)
        # Vous pourriez gérer les lignes sans correspondance (comme les messages système) ici si nécessaire

# Créer une DataFrame
df = pd.DataFrame({
    'date': pd.to_datetime(dates, format='%d/%m/%Y'),
    'time': pd.to_datetime(times, format='%H:%M').time,
    'name': names,
    'message': messages
})

# Afficher la DataFrame
print(df)


           date      time          name  \
0    2021-01-30  09:17:00         Ruben   
1    2021-01-30  09:17:00         Ruben   
2    2021-01-30  09:20:00  henri serano   
3    2021-01-30  09:20:00  henri serano   
4    2021-01-30  12:46:00         Ruben   
...         ...       ...           ...   
3662 2023-10-03  14:40:00         Ruben   
3663 2023-10-03  14:40:00  henri serano   
3664 2023-10-03  14:45:00  henri serano   
3665 2023-10-03  14:45:00         Ruben   
3666 2023-10-03  14:47:00         Ruben   

                                                message  
0                  Perso j'arrive à Paris Bercy à 17h30  
1     Tu dois partir à 17h30 de la gare (et donc par...  
2     J'arrive à Bercy à 17h30 du coup je vais arriv...  
3     Après pour le couvre feu je suis ok vu que j'a...  
4        Le souci, c'est que ça sera plus où moins fini  
...                                                 ...  
3662                      Mec t'arrive dans cb de temps  
3663               

In [3]:
def clean_message(message):
    # Enlever les emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE)
    
    message = emoji_pattern.sub(r'', message)
    
    # Enlever les liens
    message = re.sub(r'http\S+|www.\S+', '', message, flags=re.MULTILINE)
    
    # Enlever les caractères spéciaux
    message = re.sub(r'\W', ' ', message)
    
    # Enlever les caractères seuls (isolés)
    message = re.sub(r'\s+[a-zA-Z]\s+', ' ', message)
    
    # Remplacer les espaces multiples par un seul espace
    message = re.sub(r'\s+', ' ', message, flags=re.I)
    
    # Enlever les espaces en début et en fin de message
    message = message.strip()
    
    return message

# Appliquer la fonction de nettoyage aux messages
df['message'] = df['message'].apply(clean_message)

# Afficher le DataFrame après nettoyage
print(df[['message']])

                                                message
0                    Perso arrive à Paris Bercy à 17h30
1     Tu dois partir à 17h30 de la gare et donc part...
2     J arrive à Bercy à 17h30 du coup je vais arriv...
3     Après pour le couvre feu je suis ok vu que ai ...
4           Le souci est que ça sera plus où moins fini
...                                                 ...
3662                        Mec arrive dans cb de temps
3663                                          10min max
3664                    Localisation en direct partagée
3665                                       localisation
3666                 Je suis devant la sortie rue auber

[3667 rows x 1 columns]


In [13]:
import re
import markovify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Lire le fichier et structurer les données
with open("Discussion WhatsApp avec Ruben.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

questions = []
answers = []

for i in range(len(lines) - 1):
    if "Ruben:" in lines[i]:
        questions.append(lines[i].split("Ruben:")[1].strip())
        if "henri serano:" in lines[i+1]:
            answers.append(lines[i+1].split("henri serano:")[1].strip())

# Construction d'un modèle Markov à partir des réponses
text_model = markovify.NewlineText("\n".join(answers))

# 2. Utilisation de TF-IDF pour vectoriser les phrases
vectorizer = TfidfVectorizer().fit(questions)
question_vectors = vectorizer.transform(questions)

previous_exchanges = []

def get_response(user_input):
    global previous_exchanges

    # Réponse par défaut pour la confusion
    if "comprends pas" in user_input or "compris" in user_input:
        return "Désolé, je me suis mal exprimé. Peux-tu reformuler ta question ?"

    # Stocker les échanges précédents
    previous_exchanges.append(user_input)
    if len(previous_exchanges) > 6:  # Garder seulement les 3 derniers échanges (6 messages)
        previous_exchanges.pop(0)
    
    context = " ".join(previous_exchanges)
    user_vector = vectorizer.transform([context])
    
    # 3. Trouver la question la plus similaire
    similarities = cosine_similarity(user_vector, question_vectors)
    
    # 4. Générer une réponse inspirée avec Markov Chain
    response = text_model.make_short_sentence(140)  # Limiter à 140 caractères

    # Ajouter la réponse du chatbot à la liste des échanges précédents
    previous_exchanges.append(response)
    
    return response

# Tester le chatbot
while True:
    user_input = input("Vous: ")
    print("Ce que j'envoie : ",user_input)
    if user_input.lower() in ["bye", "exit", "quit"]:
        print("Chatbot: Au revoir !")
        break
    response = get_response(user_input)
    print("Chatbot:", response)


Chatbot: Je te le montrer irl mais je te dis ma vie avec tt le monde ?
Chatbot: Mdr c'est sur, mais bon au moins t'acheter à manger
Chatbot: Non je suis mort debra vient de me défoncé c'est semaine
Chatbot: Yo tu pourrais me faire avoir aussi avec hubert
Chatbot: Mais sinon ouais je vais surement rien avoir car bon aujourd'hui on va essayer de te rappeler, rappelle moi quand tu fini
Chatbot: Après je pourrais te le dirais quand on part et reste en attendant sur pariz
Chatbot: Au revoir !


In [6]:
import pandas as pd
import numpy as np
import re
import pickle
from collections import defaultdict, Counter

def create_markov_model(text, n=2):
    words = text.split()
    model = defaultdict(Counter)
    for i in range(len(words)-n):
        prefix = tuple(words[i:i+n-1])
        suffix = words[i+n-1]
        model[prefix][suffix] += 1
    return model

def generate_text(model, max_length=50):
    prefix = np.random.choice(list(model.keys()))
    output = list(prefix)
    for _ in range(max_length):
        suffixes, weights = zip(*model[prefix].items())
        suffix = np.random.choice(suffixes, p= np.array(weights)/sum(weights))
        output.append(suffix)
        prefix = tuple(output[-len(prefix):])
    return " ".join(output)

# Filtrer les messages et nettoyer le texte
ruben_text = " ".join(df[df['name'] == 'Ruben']['message'].tolist())
ruben_text = re.sub(r'[^A-Za-z\s]', '', ruben_text.lower())

# Entrainement du modèle
markov_model = create_markov_model(ruben_text, n=3)

# Sauvegarder le modèle dans un fichier .pkl
with open("markov_model.pkl", "wb") as f:
    pickle.dump(markov_model, f)


In [9]:
import pickle
import numpy as np
import re

# Charger le modèle
def load_model(file_name):
    with open(file_name, "rb") as f:
        return pickle.load(f)

def chat_with_model():
    model = load_model("markov_model.pkl")
    user_input = input("You: ")
    
    user_input = re.sub(r'[^A-Za-z\s]', '', user_input.lower())
    
    starting_words = user_input.split()[-2:]  

    model_prefix = tuple(starting_words)
    if model_prefix in model:
        response = generate_text(model, max_length=20, starting_words=starting_words)
        print("Ruben:", response)
    else:
        print("Ruben: Je ne sais pas quoi répondre à ça...")

def generate_text(model, max_length=50, starting_words=None):
    if starting_words:
        prefix = tuple(starting_words)
        output = list(prefix)
    else:
        prefix = np.random.choice(list(model.keys()))
        output = list(prefix)
    
    for _ in range(max_length):
        suffixes, weights = zip(*model[prefix].items())
        suffix = np.random.choice(suffixes, p=np.array(weights) / sum(weights))
        output.append(suffix)
        prefix = tuple(output[-len(prefix):])
    return " ".join(output)

chat_with_model()

Ruben: Je ne sais pas quoi répondre à ça...
